To Do:
* rush percentage as feature
* ewma feature instead of multiple fpts features
* L1 regularization
* Decision Tree regresssion
* sklearn time series split validation on training data, or implement this ourselves (Brian G.). Holdout data on a few future weeks. Report both the cross validation and holdout performance scores.
* look at masters theses that do this
* tune number of games in moving averages and tune number of trees in random forest

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
all_games = pd.read_csv('nfl-football-player-stats/games_1995.csv')
all_games.drop([440917], inplace=True) # delete weird Lagerrete Blount double game
all_games.loc[all_games['team'] == 'SDG', 'team'] = 'LAC'
all_games.loc[all_games['team'] == 'STL', 'team'] = 'LAR'
all_games.loc[all_games['opponent'] == 'SDG', 'opponent'] = 'LAC'
all_games.loc[all_games['opponent'] == 'STL', 'opponent'] = 'LAR'
all_games = all_games.set_index('player_id')
all_players = pd.read_csv('nfl-football-player-stats/players_1995.csv',index_col='player_id')
gamesDef = pd.read_csv('defData12-17.csv')
gamesDef.loc[gamesDef['Tm']=='SDG', 'Tm'] = 'LAC'
gamesDef.loc[gamesDef['Tm']=='STL', 'Tm'] = 'LAR'
gamesDef.loc[gamesDef['Opp']=='SDG', 'Opp'] = 'LAC'
gamesDef.loc[gamesDef['Opp']=='STL', 'Opp'] = 'LAR'
gamesDef.set_index('Tm')
gamesDef = gamesDef.drop(columns=["Rk","Time","LTime"])

In [14]:
# dictionary that has fantasy value for each nfl stat
half_ppr = {
    'rushing_yards': 0.1,
    'rushing_touchdowns': 6,
    'receiving_receptions': 0.5,
    'receiving_yards': 0.1,
    'receiving_touchdowns': 6,
    'passing_yards': 0.04,
    'passing_touchdowns': 6
}

In [4]:
# subset RB position by choosing only RBs with at least 3 fantasy points in previous game
subset_position = {
    'RB': [['rushing_attempts', 'receiving_targets'], 5.0],
    'WR': [['rushing_attempts', 'receiving_targets'], 5.0],
    'QB': [['passing_attempts'], 10.0]
}

In [5]:
def get_players_thatweek(all_games, all_players, position, year, game_number, subset_position, specific_players=None):
    if specific_players != None:
        ids = specific_players
    else:
        ids = all_players[all_players.position == position].index
        
    # return pandas df with player_id as index and player name and log draft position columns
    games = all_games.loc[ids]
    stats = subset_position[position][0]
    sum_threshold = subset_position[position][1]
    # only take RBs with 5 rush attempts or receiving targets (>= sum_threshold)
    worth_predicting = games[(games.year == year) & 
                             (games.game_number == game_number) &
                             (np.sum(games[stats], axis=1) >= sum_threshold)]
    ids = worth_predicting.index
    for_df = all_players.loc[ids, ['name', 'draft_position']]
    # players that weren't drafted give them position of last pick of draft
    for_df.fillna(255, inplace=True)
    for_df['log_draft_position'] = np.log(for_df.draft_position)
    return pd.DataFrame(data=for_df)

In [6]:
def get_team_stats(games, year, game_number):
    # returns: dataframe with team passing and rushing stats
    
    # start by getting team passing attempts
    prev_years = games[(games.year < year) &
                       (games.game_number <= 16)]
    current_year = games[(games.year == year) &
                         (games.game_number <= game_number)]
    passing_stats = pd.concat((prev_years, current_year))
    passing_stats = pd.concat((prev_years, current_year))
    passing_stats = passing_stats[['team', 'date', 'year', 'game_number', 'passing_attempts', 'passing_rating']] 
    passing_stats.sort_values(by=['year', 'game_number'], axis=0,ascending=True, inplace=True)
    passing_stats = passing_stats.groupby(['team', 'year', 'game_number'])
    # sum of pass attempts by anyone by team in year during game
    team_pass_attempts = passing_stats.passing_attempts.sum()
    # take the passing rating from the player with most pass_attempts
    pass_rating = passing_stats.apply(
        lambda x: x.nlargest(1,'passing_attempts')).droplevel(3)['passing_rating']
    ewma_pass_rating = pass_rating.groupby('team').apply(
        lambda x: x.ewm(span=16).mean()).rename('ewma_team_passing_rating')
    # Note: team pass attempts and pass_rating and ewma_pass_rating are hierarchical pd series

    # get total team rushing attempts
    games = games[(games.rushing_attempts > 0)]
    prev_years = games[(games.year < year) &
                       (games.game_number <= 16)]
    current_year = games[(games.year == year) &
                         (games.game_number <= game_number)]
    rushing_stats = pd.concat((prev_years, current_year))
    rushing_stats.sort_values(by=['year', 'game_number'], axis=0, ascending=True, inplace=True)
    rushing_stats = rushing_stats.groupby(['team', 'year', 'game_number'])
    rush_attempts = rushing_stats.rushing_attempts.sum()

    # combine all stats into one dataframe
    team_stats = pd.DataFrame(team_pass_attempts)
    team_stats = team_stats.join(pass_rating)
    team_stats = team_stats.join(ewma_pass_rating)
    team_stats = team_stats.join(rush_attempts)
    ewma_rush_attempts = team_stats.groupby('team').apply(
        lambda x: x['rushing_attempts'].ewm(span=16).mean()).droplevel(0).rename('ewma_team_rush_attempts')
    team_stats = team_stats.join(ewma_rush_attempts)
    team_stats['rush_percentage'] = team_stats.rushing_attempts/(
        team_stats.rushing_attempts+team_stats.passing_attempts)
    ewma_rush_perc = team_stats.groupby('team').apply(
        lambda x: x['rush_percentage'].ewm(span=16).mean()).droplevel(0).rename('ewma_team_rush_percentage')
    team_stats = team_stats.join(ewma_rush_perc)
    team_stats = team_stats.sort_values(by=['year', 'game_number'],axis=0,ascending=False).groupby('team').nth([0])
    return team_stats

In [7]:
def get_def_data(gamesDef, year, gameNumber, gamesBack = 5):
    # return def stats for all teams using last 5 weeks as Pandas Dataframe
    prevSeasonDef = gamesDef[gamesDef['Year']==(year-1)]
    currentSeasonDef = gamesDef[(gamesDef['Year']==year) & (gamesDef['Game']<=gameNumber)]
    df = pd.concat((prevSeasonDef, currentSeasonDef))
    df.sort_values(by=['Year', 'Game'], axis=0, ascending=False, inplace=True)
    grouped = df.groupby('Tm', sort=False)
    def_stats = grouped.nth(list(range(0,gamesBack))).groupby('Tm', sort=False).mean()[['DY/P', 'TO']]
    def_stats.index.rename('opp', inplace=True)
    return def_stats

In [9]:
df2 = get_team_stats(all_games, 2016, 1)
df2

,passing_attempts,passing_rating,ewma_team_passing_rating,rushing_attempts,ewma_team_rush_attempts,rush_percentage,ewma_team_rush_percentage
team,,,,,,,
ARI,24,104.7,98.388859,19,26.432704,0.441860,0.535140
ATL,27,112.6,93.186236,22,24.726216,0.448980,0.486039
BAL,22,100.3,81.982241,28,24.401527,0.560000,0.483403
BUF,15,79.9,92.942911,24,31.606905,0.615385,0.638627
CAR,18,69.5,99.807725,32,31.866786,0.640000,0.621057
CHI,16,76.2,90.729656,20,27.198219,0.555556,0.566272
CIN,23,114.0,104.756530,19,28.112698,0.452381,0.570310
CLE,12,55.0,71.885403,21,24.292479,0.636364,0.540958
DAL,25,69.4,78.167370,30,25.301004,0.545455,0.548089


In [10]:
# example usage
df1 = get_def_data(gamesDef, 2016, 1, gamesBack=15)
df1

,DY/P,TO
opp,,
OAK,5.350027,2.100000
NOR,6.745927,1.700000
IND,5.722467,2.444444
PIT,5.646320,2.153846
NYJ,5.128120,1.916667
DET,5.643000,2.111111
WAS,6.248087,1.909091
ATL,5.584240,2.545455
LAC,6.014253,1.636364


In [16]:
def get_features_response(players, all_games, year, game_number, points_dict, defWeeksBack=5):
    games = all_games.loc[players.index]
    
    # compute fpts for each row
    games['fpts'] = games['game_number']*0
    for stat, value in zip(points_dict.keys(), points_dict.values()):
        games['fpts'] = games['fpts'] + games[stat]*value
    
    prev_years = games[(games.year < year) &
                       (games.game_number <= 16)]
    current_year = games[(games.year == year) &
                         (games.game_number <= game_number)]
    #current_game = games[(games.year == year) &
    #                 (games.game_number==game_number)]
    next_game = games[(games.year == year) &
                     (games.game_number == game_number+1)]
    df = pd.concat((prev_years, current_year))
    # only keep columns needed for indiviual statistics
    df = df[['team', 'date', 'year', 'game_number', 'fpts', 'rushing_attempts', 'receiving_targets']]
    df.sort_values(by=['player_id', 'year', 'game_number'], axis=0,
                   ascending=True, inplace=True)
    
    ## Individual Statistics
    # group dataframe by index
    df['num_games'] = df.groupby(df.index).cumcount().rename('num_games') + 1
    df['log_num_games'] = np.log(df.num_games)
    df['ewma_rushing_attempts'] = df.groupby(df.index).apply(
        lambda x: x['rushing_attempts'].ewm(span=16).mean()).droplevel(0).rename('ewma_rushing_attempts')
    df['ewma_receiving_targets'] = df.groupby(df.index).apply(
        lambda x: x['receiving_targets'].ewm(span=16).mean()).droplevel(0).rename('ewma_receiving_targets')
    df['ewma_fpts'] = df.groupby(df.index).apply(
        lambda x: x['fpts'].ewm(span=16).mean()).droplevel(0).rename('ewma_fpts')
    df = df[['team', 'date', 'year', 'log_num_games', 'game_number', 'fpts', 'ewma_fpts', 'ewma_rushing_attempts', 'ewma_receiving_targets']]
    df = df.sort_values(by=['year','game_number'],axis=0,ascending=False).groupby('player_id').nth([0])

    ## Team Statistics
    # get rush_percentage, rushing_attempts, and passer rating of teams
    team_stats = get_team_stats(games = all_games, year=year, game_number=game_number)
    # join team stats to df by team
    df = df.reset_index().join(team_stats, how='left', on='team').set_index('player_id')#.drop_duplicates()
    
    # get defenseive stats
    def_stats = get_def_data(gamesDef, year, game_number, defWeeksBack)
    #def_stats.reset_index(inplace=True)
    df['opp'] = next_game['opponent']
    df['next_fpts'] = next_game['fpts']
    # Response variable is the fantasy points of the next game, NA should be zero because they
    # didn't play or didn't score
    df['next_fpts'] = df['next_fpts'].fillna(0)
    df = df.join(def_stats, how='left', on='opp')
    df.sort_values(by=['player_id', 'year', 'game_number'], axis=0,
                   ascending=True, inplace=True)
    # join df to players with name and draft info
    df = players.join(df)
    
    # drop players who don't play in the next week
    df.dropna(subset=['opp'], inplace=True)
    
    #df['ros_ppg'] = rest_year.groupby('player_id').fpts.mean().rename('ros_ppg')
    #ros_games = rest_year.groupby('player_id').fpts.count().rename('ros_games', inplace=True)
    return df

In [17]:
# example usage
# train to game number 
a = get_players_thatweek(all_games, all_players,'QB',2015,15,subset_position)
get_features_response(a, all_games, 2015, 15, half_ppr)

,name,draft_position,log_draft_position,team,date,year,log_num_games,game_number,fpts,ewma_fpts,...,passing_rating,ewma_team_passing_rating,rushing_attempts,ewma_team_rush_attempts,rush_percentage,ewma_team_rush_percentage,opp,next_fpts,DY/P,TO
player_id,,,,,,,,,,,,,,,,,,,,,
6975,Ryan Fitzpatrick,250.0,5.521461,NYJ,2015-12-27,2015,4.718499,15,31.14,24.556966,...,109.4,93.628569,27,28.829277,0.509434,0.557665,BUF,20.44,6.14284,2.000000
14490,A.J. McCarron,164.0,5.099866,CIN,2015-12-28,2015,1.791759,15,16.10,10.586326,...,87.8,103.473943,33,29.638088,0.600000,0.582354,BAL,18.60,5.24526,2.250000
7476,Blaine Gabbert,10.0,2.302585,SFO,2015-12-27,2015,3.583519,15,21.90,16.545870,...,106.2,85.136030,23,23.158932,0.511111,0.532043,LAR,20.86,5.80030,2.000000
16905,Brock Osweiler,57.0,4.043051,DEN,2015-12-28,2015,3.044522,15,18.46,13.110358,...,100.3,86.671302,21,26.230159,0.437500,0.519667,LAC,15.28,5.09350,1.750000
24421,Russell Wilson,75.0,4.317488,SEA,2015-12-27,2015,4.143135,15,27.36,28.718668,...,88.4,110.612540,22,31.597021,0.468085,0.601360,ARI,27.18,5.32340,2.000000
11999,Case Keenum,255.0,5.541264,LAR,2015-12-27,2015,2.708050,15,10.52,12.883541,...,86.0,81.684873,30,26.562704,0.681818,0.605162,SFO,9.44,5.14754,2.333333
19319,Matt Ryan,3.0,1.098612,ATL,2015-12-27,2015,4.828314,15,18.34,18.905485,...,119.6,88.917432,30,24.968340,0.566038,0.487111,NOR,25.36,6.66166,1.000000
4993,Jay Cutler,11.0,2.397895,CHI,2015-12-27,2015,4.890349,15,14.34,19.557563,...,100.2,92.022536,39,28.712380,0.661017,0.565345,DET,22.90,5.21112,1.000000
13917,Ryan Mallett,74.0,4.304065,BAL,2015-12-27,2015,2.639057,15,16.96,8.422279,...,95.0,82.211856,31,24.311295,0.525424,0.481380,CIN,18.08,5.04180,1.500000


In [18]:
## Pretend its 2016, nfl week 5

# Get all features and responses for Running Backs from START_YEAR to YEAR before GAME_NUMBER
POSITION = 'QB'
START_YEAR = 2014
YEAR = 2017
GAME_NUMBER = 11
# features 
FEATURES = ['name', 'team', 'date', 'year', 'game_number',
            'log_draft_position','log_num_games', 'ewma_fpts', 
            'ewma_team_rush_attempts', 'ewma_team_rush_percentage', 
            'ewma_team_passing_rating', 'ewma_rushing_attempts', 'ewma_receiving_targets',
            'DY/P', 'TO', 'next_fpts']
# response
RESPONSE = ['next_fpts']

# append features and response each week to these lists
feature_list = []
response_list = []

for train_year in range(START_YEAR, YEAR+1):

    # if current year don't go past nfl week
    if train_year == YEAR:
        game_limit = GAME_NUMBER
    else: # if previous year don't go past regular season (predict game number 16)
        game_limit = 15

    for train_week in range(1,game_limit+1):
        players = get_players_thatweek(all_games, all_players, POSITION, train_year, train_week, subset_position)
        train = get_features_response(players, all_games, train_year, train_week, points_dict=half_ppr, defWeeksBack=5)
        print('Got train data for year ' +str(train_year) + ' game number ' + str(train_week))
        print('Count of NAN: ' + str(train.isnull().sum()))
        feature = train[FEATURES]
        #response = train[RESPONSE]
        feature_list.append(feature)
        #response_list.append(response)

## After all feature and response training lists have been created
#  Concat lists into train dataframes
train_x = pd.concat(feature_list)
#train_y = pd.concat(response_list)

# fill NaN in train_x with zeros
#train_x = train_x.fillna(0)

Got train data for year 2014 game number 1
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year                         0
log_num_games                0
game_number                  0
fpts                         0
ewma_fpts                    0
ewma_rushing_attempts        0
ewma_receiving_targets       0
passing_attempts             0
passing_rating               0
ewma_team_passing_rating     0
rushing_attempts             0
ewma_team_rush_attempts      0
rush_percentage              0
ewma_team_rush_percentage    0
opp                          0
next_fpts                    0
DY/P                         0
TO                           0
dtype: int64
Got train data for year 2014 game number 2
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year 

Got train data for year 2014 game number 12
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year                         0
log_num_games                0
game_number                  0
fpts                         0
ewma_fpts                    0
ewma_rushing_attempts        0
ewma_receiving_targets       0
passing_attempts             0
passing_rating               0
ewma_team_passing_rating     0
rushing_attempts             0
ewma_team_rush_attempts      0
rush_percentage              0
ewma_team_rush_percentage    0
opp                          0
next_fpts                    0
DY/P                         0
TO                           0
dtype: int64
Got train data for year 2014 game number 13
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
yea

Got train data for year 2015 game number 8
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year                         0
log_num_games                0
game_number                  0
fpts                         0
ewma_fpts                    0
ewma_rushing_attempts        0
ewma_receiving_targets       0
passing_attempts             0
passing_rating               0
ewma_team_passing_rating     0
rushing_attempts             0
ewma_team_rush_attempts      0
rush_percentage              0
ewma_team_rush_percentage    0
opp                          0
next_fpts                    0
DY/P                         0
TO                           0
dtype: int64
Got train data for year 2015 game number 9
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year 

Got train data for year 2016 game number 4
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year                         0
log_num_games                0
game_number                  0
fpts                         0
ewma_fpts                    0
ewma_rushing_attempts        0
ewma_receiving_targets       0
passing_attempts             0
passing_rating               0
ewma_team_passing_rating     0
rushing_attempts             0
ewma_team_rush_attempts      0
rush_percentage              0
ewma_team_rush_percentage    0
opp                          0
next_fpts                    0
DY/P                         0
TO                           0
dtype: int64
Got train data for year 2016 game number 5
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year 

Got train data for year 2016 game number 15
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year                         0
log_num_games                0
game_number                  0
fpts                         0
ewma_fpts                    0
ewma_rushing_attempts        0
ewma_receiving_targets       0
passing_attempts             0
passing_rating               0
ewma_team_passing_rating     0
rushing_attempts             0
ewma_team_rush_attempts      0
rush_percentage              0
ewma_team_rush_percentage    0
opp                          0
next_fpts                    0
DY/P                         0
TO                           0
dtype: int64
Got train data for year 2017 game number 1
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year

Got train data for year 2017 game number 11
Count of NAN: name                         0
draft_position               0
log_draft_position           0
team                         0
date                         0
year                         0
log_num_games                0
game_number                  0
fpts                         0
ewma_fpts                    0
ewma_rushing_attempts        0
ewma_receiving_targets       0
passing_attempts             0
passing_rating               0
ewma_team_passing_rating     0
rushing_attempts             0
ewma_team_rush_attempts      0
rush_percentage              0
ewma_team_rush_percentage    0
opp                          0
next_fpts                    0
DY/P                         0
TO                           0
dtype: int64


In [20]:
len(train_x)

1645

In [21]:
train_x.to_csv('/Users/nickvarberg/Desktop/School/Football-Prediction/qbs_2014_2017.csv')

In [12]:
lr = None
r_sqr_list = []
mean_abs_err_list = []
mean_test_fpts_list = []

# train mlr if not trained yet
if lr is None:

    # Train a linear regression model to predict rest of season ppg for RBs 
    lr = LinearRegression()
    lr = lr.fit(train_x, train_y)
    print('Model trained. Here are the results:')
    print('Intercept:')
    print(lr.intercept_)
    for feat, coef in zip(FEATURES, lr.coef_):
        print(feat + ' ' + str(coef))


Got train data for year 2016 game number 1
Got train data for year 2016 game number 2
Got train data for year 2016 game number 3
Got train data for year 2016 game number 4
train data: 
           log_draft_position  log_num_games  ewma_fpts  \
player_id                                                 
17858                4.836282       4.077537   8.912268   
10586                4.043051       3.091042   9.707592   
5943                 5.541264       3.737670   6.088254   
11664                4.553877       2.639057   7.504538   
23267                5.267858       2.639057  10.262829   

           ewma_team_rush_attempts  ewma_team_rush_percentage  \
player_id                                                       
17858                    27.739056                   0.559673   
10586                    26.397092                   0.547583   
5943                     26.397092                   0.547583   
11664                    25.430036                   0.500759   
23267      

In [13]:
# test linear regression
for test_week in range(GAME_NUMBER,8):
    # Get test_x and test_y
    players = get_players_thatweek(all_games, all_players, POSITION, YEAR, test_week, subset_position)
    test = get_features_response(players, all_games, YEAR, test_week, points_dict=half_ppr)
    test_x = test[FEATURES]
    test_y = test[RESPONSE]

    # fill NaN in test_x with zeros
    test_x = test_x.fillna(0)

    # Score gives R^2 of prediction of test_x wrt test_y
    # Note: R^2 is correlated to how many easy predictions
    #  (players with low fantasy points are easy to predict).
    #  R^2 decreases when predicting on fewer but better running backs.
    print('Year: ' +str(YEAR) + '  Game Number: '+ str(test_week))
    r_sqr = lr.score(test_x, test_y)
    r_sqr_list.append(r_sqr)
    mean_abs_err = round(mean_absolute_error(test_y, lr.predict(test_x)),2)
    mean_abs_err_list.append(mean_abs_err)
    mean_test_fpts = np.mean(test_y)
    mean_test_fpts_list.append(mean_test_fpts)
    
    print('R^2: ' + str(round(r_sqr,2)))
    print('Mean Abs Error: ' + str(mean_abs_err))

print('Mean R^2: ' + str(round(np.mean(r_sqr_list), 2)))
print('Overall mean abs err: ' + str(round(np.mean(mean_abs_err_list), 2)))
print('Overall mean test fpts:' + str(round(np.mean(mean_test_fpts_list),2)))

Year: 2016  Game Number: 5
R^2: 0.14
Mean Abs Error: 6.44
Year: 2016  Game Number: 6
R^2: 0.14
Mean Abs Error: 6.51
Year: 2016  Game Number: 7
R^2: 0.16
Mean Abs Error: 5.52
Mean R^2: 0.14
Overall mean abs err: 6.16
Overall mean test fpts:9.25


In [ ]:
print('number of running back games: ' + str(len(train_x)))